In [1]:
import numpy as np
from astropy.time import Time
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.dates as mdates
from ipywidgets import interact

In [2]:
######################################
dir_files = '../DVA/Data_Files/Dec13_azimuth_scans/'
out_dir = '../DVA/Data_Files/Dec13_azimuth_scans/'
date = '2021-12-13'
hr_start = '17'
hr_end = '24'
######################################

hr1 = int(hr_start)
hr2 = int(hr_end)
outname = 'data_'+date+'_'+f"{hr1:02}"+'_to_'+f"{hr2:02}"

# Read in the data, frequency, time arrays:
RR_set,LL_set,reRL_set,imRL_set = np.load(dir_files+outname+'.npy')
freq_set = np.load(dir_files+outname+'_freq.npy')
time_set = np.load(dir_files+outname+'_time.npy')
# Read in the position arrays (Note: do not use ra and dec yet - 
# these are not correct in the files)
#ra_set,dec_set,az_set,el_set = np.load(dir_files+outname+'_coords.npy')

t_fix = Time(time_set, format='isot',scale='utc')
t_plt = t_fix.plot_date

### Choose a frequency !!

In [3]:
fr_plot = 850

In [4]:
window1_start = 10750
window1_end = 11200
print("Window 1: \n", t_fix[window1_start], t_fix[window1_end], '\n')

window2_start = 13600
window2_end = 14250
print("Window 2: \n", t_fix[window2_start], t_fix[window2_end], '\n')

df = freq_set[1]-freq_set[0]
w = np.where( abs(freq_set-fr_plot) < df/2.)[0][0]
print("Frequency Chosen: ",freq_set[w])
print("w:", w)

t_plt1 = t_plt[window1_start:window1_end]
reRL_set1 = reRL_set[window1_start:window1_end,w]
imRL_set1 = imRL_set[window1_start:window1_end,w]
polar1 = []
for i,j in zip(reRL_set1,imRL_set1):
    PI = np.sqrt((i**2)+(j**2))
    polar1.append(PI)
plr_plot1 = np.array(polar1)

t_plt2 = t_plt[window2_start:window2_end]
reRL_set2 = reRL_set[window2_start:window2_end,w]
imRL_set2 = imRL_set[window2_start:window2_end,w]
polar2 = []
for i,j in zip(reRL_set2,imRL_set2):
    PI = np.sqrt((i**2)+(j**2))
    polar2.append(PI)
plr_plot2 = np.array(polar2)
# print(polar2)

Window 1: 
 2021-12-13T20:56:36.532 2021-12-13T21:06:38.447 

Window 2: 
 2021-12-13T22:00:29.797 2021-12-13T22:15:37.952 

Frequency Chosen:  850.0
w: 4725


### Polarized Intensity:

In [5]:
polarized = []
reRL_set_use = reRL_set[:,w]    #TODO: maybe I can try making something general that I can scan over all frequencies
imRL_set_use = imRL_set[:,w]
for i,j in zip(reRL_set_use,imRL_set_use):
    PI = np.sqrt((i**2)+(j**2))
    polarized.append(PI)
polarized_plot = np.array(polarized)

In [6]:
# in_power_array        Input power array (RR, LL, reRL, imRL) at a single frequency
# numt                  Number of time points to include in the 'window' in which the standard deviation is calculated
# threshold             Array of standard deviation thresholds to use in each stage of the RFI excision 

def Is_RFI(in_power_array, threshold, idx, interval_start, interval_end):
    retVal = False
    in_power_mean = np.nanmean(in_power_array[interval_start:interval_end])
    in_power_std = np.nanstd(in_power_array[interval_start:interval_end])
    if abs(in_power_array[idx]) > abs(in_power_mean)+threshold*in_power_std:
        retVal = True
    return retVal
    
def Find_Slope(array, idx):
    return array[idx+2] - array[idx]

def Get_RFI_Duration(in_power_array, interval_start, interval_end):  #NOTE: This approach might have issues if the interval has a nan on it
    #Find the first slope
    first_slope = Find_Slope(in_power_array, interval_start)
    #Initiate the RFI interval indexes at the boundaries of the set
    RFI_start = interval_start
    RFI_end = interval_end
    #Initiate as the fastest rise & fall
    steepest_rise = first_slope
    steepest_fall = first_slope 
    start_idx = interval_start + 1  # Ignores the first slope (since it's already calculated)
    end_idx = interval_end - 2      # Avoids overflowing (since each slope is calculated 2 indexes ahead)
    for idx in range(start_idx, end_idx):
        current_slope = Find_Slope(in_power_array, idx)
        if(current_slope >  steepest_rise):
            steepest_rise = current_slope
            RFI_start = idx
        elif(current_slope < steepest_fall):
            steepest_fall = current_slope
            RFI_end = idx + 2
    return (RFI_end - RFI_start)


def Get_RFI_Excision_Mask(time_array,in_power_array,time_steps,threshold):
    mask = np.zeros([len(time_array)])  #Initiate empty mask
    start_idx = time_steps
    end_idx = len(time_array)-time_steps              
    for idx in range(start_idx, end_idx):
        if Is_RFI(in_power_array, threshold, idx, idx - time_steps, idx + time_steps):
            RFI_duration = Get_RFI_Duration(in_power_array, idx - time_steps, idx + time_steps)
            mask[idx - RFI_duration:idx + RFI_duration] = 1
    return(mask)

def Apply_RFI_Excision_Mask(mask, input_array):
    masked_copy = input_array.copy()
    for idx in range(0, len(input_array)-1):
        if(mask[idx] == 1):
            masked_copy[idx] = np.nan
    return(masked_copy)



In [7]:
df = freq_set[1]-freq_set[0]
w = np.where( abs(freq_set-fr_plot) < df/2.)[0][0]
print(freq_set[w])

time_steps = 30
threshold = 4

polarized_mask = Get_RFI_Excision_Mask(t_plt,polarized_plot,time_steps,threshold)
RR_with_polarized_mask = Apply_RFI_Excision_Mask(polarized_mask, RR_set[:,w])
polarized_plot_masked = Apply_RFI_Excision_Mask(polarized_mask, polarized_plot)


850.0


In [8]:
def Interactive_Plotter2(x, x_zoom, y, y_zoom):
    fig,axs = plt.subplots(1,1,figsize=(20,6))    
    axs.set_ylim(y*1e7,y*1e7+y_zoom*1e7)
    axs.plot(t_plt,RR_set[:,w], label = 'RR RFI')
    # axs.plot(t_plt,ymasked_plt, label = 'RR - AO')
    axs.plot(t_plt, RR_with_polarized_mask, label = 'RR with polarized mask')
    axs.plot(t_plt,polarized_plot, label = 'Polarized Intensity RFI')
    axs.plot(t_plt,polarized_plot_masked, label = 'Polarized Intensity')
    zoom_sensitivity = 1.2
    x_window = int(len(t_plt)/(x_zoom**(1/zoom_sensitivity)))
    if(x_zoom == 1):                                            #Plots the entire set
        axs.set_xlim(t_plt[0], t_plt[len(t_plt)-1])         
    elif(x + x_window <= len(t_plt) -1):                        #Stops from overflowing the right-most limit
        axs.set_xlim(t_plt[x], t_plt[x + x_window])
    else:
        axs.set_xlim(t_plt[x-x_window], t_plt[len(t_plt)-1])    #Plots between x and x+x_window

    axs.grid()
    axs.legend()
    axs.set_title("RFI Excision Test")
    axs.set_xlabel("Time (UTC)")
    axs.set_ylabel("Power")
    axs.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    axs.fmt_xdata = mdates.DateFormatter('%H:%M:%S')

interact(Interactive_Plotter2, x=(0,len(t_plt)-1), x_zoom=(1,150), y=(0,6), y_zoom=(3,15, 0.05))


interactive(children=(IntSlider(value=9474, description='x', max=18949), IntSlider(value=75, description='x_zo…

<function __main__.Interactive_Plotter2(x, x_zoom, y, y_zoom)>